#Steam Game Recommended Project

In [1]:
#Imports Library
import pandas as pd
import numpy as np
import os
import pickle

In [7]:
# os.environ['KAGGLE_CONFIG_DIR']='/workspaces/codespaces-jupyter/key'

In [8]:
# !kaggle datasets download -d fronkongames/steam-games-dataset

In [9]:
# !unzip /workspaces/codespaces-jupyter/notebooks/steam-games-dataset.zip -d game

In [2]:
games_data=pd.read_csv('/workspaces/codespaces-jupyter/notebooks/game.csv')

In [5]:
games_data=games_data[['AppID','Name','About the game','Categories','Genres','Tags']]

In [6]:
pickle.dump(games_data,open('game_data.pkl','wb'))

In [12]:
# games_data.isnull().sum()

In [13]:
# games_data['Release date']=games_data['Release date'].astype(dtype='datetime64')

In [14]:
# games_data.sort_values(by='Release date',ascending=False,inplace=True) 

In [15]:
# pure_data=games_data[['AppID','Name','About the game','Categories','Genres','Tags','Header image']].dropna()

In [16]:
# pure_data

In [17]:
# games_data=pure_data[['AppID','Name','About the game','Categories','Genres','Tags']]

In [18]:
# image_data=pure_data[['AppID','Header image']]

In [19]:
# games_data

In [20]:
# iamge_data.isnull().sum()

In [21]:
# games_data.isnull().sum()

In [22]:
# pure_data.isnull().sum()

In [23]:
# index = pd.Index(range(0, 58514 , 1))
# pure_data = pure_data.set_index(index)

In [24]:
# pure_data

In [25]:
# games_data=pure_data[['AppID','Name','About the game','Categories','Genres','Tags']]

In [26]:
# games_data

In [27]:
# image_data=pure_data[['AppID','Header image']]

In [28]:
# image_data

In [29]:
# games_data=games_data[0:18560]
# image_data=image_data[0:18560]

In [30]:
# games_data

In [31]:
# image_data.to_csv('image_data.csv')

In [32]:
# image_data.to_pickle(open('image_data','wb'))

In [33]:
# games_data.to_csv('game.csv')

In [34]:
# games_data.info()

#Column i need for recommends games to play
- AppID
- Name
- About the game
- Publishers
- Categories                      
- Genres                          
- Tags

In [35]:
new_games_data=pd.DataFrame(games_data)

In [36]:
# new_games_data.isnull().sum()

In [37]:
# new_games_data.shape

In [38]:
new_games_data['Genres']=new_games_data['Genres'].apply(lambda x:x.split())

In [39]:
new_games_data['Categories']=new_games_data['Categories'].apply(lambda x:x.split())

In [40]:
new_games_data['Tags']=new_games_data['Tags'].apply(lambda x:x.split())

In [41]:
new_games_data['About the game']=new_games_data['About the game'].apply(lambda x:x.split())

In [42]:
new_games_data['Categories']=new_games_data['Categories'].apply(lambda x:[i.replace(" ","") for i in x])

In [43]:
new_games_data['Genres']=new_games_data['Genres'].apply(lambda x:[i.replace(" ","") for i in x])

In [44]:
new_games_data['Tags']=new_games_data['Tags'].apply(lambda x:[i.replace(" ","") for i in x])

In [45]:
# new_games_data['About the game'][0]

In [46]:
# new_games_data

In [47]:
new_games_data['tags']=new_games_data['About the game'] + new_games_data['Categories'] + new_games_data['Genres'] + new_games_data['Tags']

In [48]:
new_df=new_games_data[['AppID','Name','tags']]

In [49]:
# new_df

In [50]:
new_df['tags']=new_df['tags'].apply(lambda x:" ".join(x))

/tmp/ipykernel_100089/487797088.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['tags']=new_df['tags'].apply(lambda x:" ".join(x))


In [51]:
new_df['tags']=new_df['tags'].apply(lambda x:x.lower())

/tmp/ipykernel_100089/4224080999.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['tags']=new_df['tags'].apply(lambda x:x.lower())


In [52]:
# new_df

In [53]:
from nltk.stem.porter import PorterStemmer
ps= PorterStemmer()

In [54]:
def stem(text):
  y=[]
  for i in text.split():
    y.append(ps.stem(i))
  return " ".join(y)

In [55]:
new_df['tags']=new_df['tags'].apply(stem)

/tmp/ipykernel_100089/3514595201.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['tags']=new_df['tags'].apply(stem)


In [56]:
from sklearn.feature_extraction.text import CountVectorizer

In [57]:
cv=CountVectorizer(max_features=5000,stop_words='english')

In [58]:
vactors=cv.fit_transform(new_df['tags']).toarray()

In [59]:
# cv.get_feature_names_out()

In [60]:
from sklearn.metrics.pairwise import cosine_similarity

In [61]:
similarity=cosine_similarity(vactors)

In [62]:
def recommend(game):
    index = new_df[new_df['Name'] == game].index[0]
    distances = sorted(list(enumerate(similarity[index])),reverse=True,key = lambda x: x[1])
    for i in distances[1:6]:
        print(new_df.iloc[i[0]].Name)

In [65]:
recommend('Stray')

The Door in the Basement
Ever Seen A Cat? 2
A Castle Full of Cats
IN THE BUILDING: CATS
Catsperience


In [64]:
pickle.dump(similarity,open('similarity.pkl','wb'))